### ML for Flask

In [12]:
import pickle
import pandas as pd
import numpy as np
xgb_model_min_loaded = pickle.load(open('ny_min_xgb_model_min.pickle', "rb"))
xgb_model_max_loaded = pickle.load(open('ny_max_xgb_model_min.pickle', "rb"))


sel_features = ['rating','.Net', 'AI', 'AWS', 'Azure', 'Big Data', 'Business Intelligence',
                'C/C++', 'Data Analysis', 'Data Science', 'Data Warehouse', 'Excel',
                'Git', 'HBase', 'Hadoop', 'Hive', 'Java', 'JavaScript', 'Kafka',
                'Linux', 'MATLAB', 'Machine Learning', 'Microsoft Office',
                'Microsoft SQL Server', 'MongoDB', 'MySQL', 'Natural Language Processing',
                'NoSQL', 'Oracle', 'Perl', 'Pig', 'PostgreSQL', 'Project Management',
                'Python', 'R', 'S3', 'SAS', 'SPSS', 'SQL', 'Scala', 'Scripting',
                'Shell Scripting', 'Software Development', 'Spark', 'Tableau', 'TensorFlow']

def NYC_salary_with_skills(rating,inputs_list):
    sample_list = [0] * (len(sel_features))
    input_X = pd.DataFrame([sample_list],
                               columns=list(sel_features))
    for inputs in inputs_list:
        if inputs in list(input_X.columns):
            input_X[inputs] = 1
    input_X['rating'] = float(rating)
    salary = (int(xgb_model_min_loaded.predict(input_X[:1])), int(xgb_model_max_loaded.predict(input_X[:1])))
    return salary

In [14]:
skill_money = {}
skill_money_list = []
for skill in sel_features[1:]:
    skill_money['skill'] = skill
    skill_money['salary'] = NYC_salary_with_skills(3.6, [skill])
    skill_money_list.append(skill_money)
    skill_money = {}
single_skill = pd.DataFrame(skill_money_list).sort_values('salary', ascending=False)

In [15]:
def NYC_salary_with_skills_and(rating, inputs_list):
    
    sample_list = [0] * (len(sel_features))
    input_x = pd.DataFrame([sample_list],
                 columns =list(sel_features))

    for inputs in inputs_list:
        if inputs in list(input_x.columns):
            input_x[inputs] = 1
    salary_min = int(xgb_model_min_loaded.predict(input_x[:1]))
    salary_max = int(xgb_model_max_loaded.predict(input_x[:1]))

    
    # suggest skill with more salary    
    suggest_list = []
    all_list = inputs_list

    for skill in single_skill['skill'][:10]:
        if skill not in inputs_list:
            suggest_list.append(skill)

        suggest_list_salary = {}
        suggest_list_salary_list = []
        for skill in suggest_list:
            all_list = []
            all_list = inputs_list+ [skill]
            suggest_list_salary['skill'] = skill
            suggest_list_salary['salary'] = int(np.subtract(NYC_salary_with_skills(rating,all_list), NYC_salary_with_skills(rating,inputs_list)).mean())
            suggest_list_salary_list.append(suggest_list_salary)
            suggest_list_salary = {}
    suggest_skills = pd.DataFrame(suggest_list_salary_list).sort_values('salary',ascending=False)
    suggest_skills = suggest_skills[suggest_skills['salary'] > 0 ]
    suggest_skills.columns = ['Skill', 'Salary increase by $']
    return {'Min_Salary': salary_min, 'Max_Salary': salary_max, 'Suggest_Skills': suggest_skills}

In [16]:
NYC_salary_with_skills_and(3.6,['MySql','R','Tableau','Python'])

{'Min_Salary': 66400,
 'Max_Salary': 75639,
 'Suggest_Skills':                   Skill  Salary increase by $
 4    Project Management                 28649
 0          Data Science                 22929
 1  Microsoft SQL Server                 21086
 2                    AI                 17598
 3            TensorFlow                 13513
 8                MATLAB                 11232
 5                  Hive                  7607
 6                   SAS                  1298}

### Viz for Flask

In [21]:
input_list = ['MySql','R','Tableau','Python']
rating = 3.6

In [22]:
NYC_salary_with_skills_and(rating,input_list)

{'Min_Salary': 66400,
 'Max_Salary': 75639,
 'Suggest_Skills':                   Skill  Salary increase by $
 4    Project Management                 28649
 0          Data Science                 22929
 1  Microsoft SQL Server                 21086
 2                    AI                 17598
 3            TensorFlow                 13513
 8                MATLAB                 11232
 5                  Hive                  7607
 6                   SAS                  1298}

In [23]:
import pandas as pd
import numpy as np
import pyecharts
from pyecharts import options as opts
df = pd.read_csv('skills_ranking.csv',header=None)

In [25]:
df.columns = ['skill','count']
df

,skill,count
0,Python,3325
1,SQL,3104
2,Machine Learning,2297
3,R,2234
4,Hadoop,1714
...,...,...
459,TestNG,1
460,UltiPro,1
461,Wireframing,1
462,Xero,1
